In [58]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import torch.nn as nn
from collections import OrderedDict

In [59]:
# Initialize
class Config:
    USE_GPU = True
    USE_RANDOM_SPLIT = False
    USE_DATAPARALLEL = True
    SEED = 42
    NUM_EPOCHS = 30
    LEARNING_RATE = 0.00001
    BATCH_SIZE = 128
    IMG_HEIGHT = {5: 32, 20: 64, 60: 96}
    IMG_WIDTH = {5: 15, 20: 60, 60: 180}
    IMG_CHANNELS = 1
    
    # Early stopping
    EARLY_STOPPING_PATIENCE = 2  
    MIN_DELTA = 0.00001

In [60]:
IMG_HEIGHT_SELECTED = 32
IMG_WIDTH_SELECTED = 15

class CNN5d(nn.Module):
    def init_weights(self, m):
        if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
            torch.nn.init.xavier_uniform_(m.weight)  # 修正
            m.bias.data.fill_(0.01)
    
    def __init__(self):
        super(CNN5d, self).__init__()
        self.conv1 = nn.Sequential(OrderedDict([
            ('Conv', nn.Conv2d(1, 64, (5, 3), padding=(2, 1), stride=(1, 1), dilation=(1, 1))),
            ('BN', nn.BatchNorm2d(64, affine=True)),
            ('ReLU', nn.ReLU()),
            ('Max-Pool', nn.MaxPool2d((2, 1)))
        ]))
        self.conv1 = self.conv1.apply(self.init_weights)
        
        self.conv2 = nn.Sequential(OrderedDict([
            ('Conv', nn.Conv2d(64, 128, (5, 3), padding=(2, 1), stride=(1, 1), dilation=(1, 1))),
            ('BN', nn.BatchNorm2d(128, affine=True)),
            ('ReLU', nn.ReLU()),
            ('Max-Pool', nn.MaxPool2d((2, 1)))
        ]))
        self.conv2 = self.conv2.apply(self.init_weights)
        
        # 計算攤平大小
        dummy_input = torch.zeros(1, 1, IMG_HEIGHT_SELECTED, IMG_WIDTH_SELECTED)
        flattened_size = self.conv2(self.conv1(dummy_input)).view(1, -1).shape[1]

        self.DropOut = nn.Dropout(p=0.5)
        self.FC = nn.Linear(flattened_size, 2)
        self.FC.apply(self.init_weights)

    def forward(self, x): 
        # 輸入數據應為 [N, 32, 15]
        if x.ndim == 4:  # 若有多餘維度，去掉
            x = x.squeeze(1)
        x = x.unsqueeze(1).to(torch.float32)  # 增加通道維度，變為 [N, 1, 32, 15]
        x = self.conv1(x)  # 通過卷積層
        x = self.conv2(x)  # 通過第二層卷積
        x = self.DropOut(x.view(x.shape[0], -1))  # 攤平
        x = self.FC(x)  # 全連接層
        return x

In [61]:
# 定義加載模型的函數
def load_model_for_prediction(filename='best_model.pth'):
    model = CNN5d()  # 初始化模型結構
    checkpoint = torch.load(filename, map_location=torch.device('cpu'))
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    model.eval()  # 設置為評估模式
    return model

In [62]:
# 定義預測函數
def predict_new_data(model, image_paths):
    predictions = []
    probabilities = []

    for image_path in image_paths:
        try:
            # 加載圖片並轉為 Tensor
            image = Image.open(image_path).convert('1')  # 保留二值圖像
            image_tensor = torch.tensor(np.array(image, dtype=np.float32))  # 轉為浮點數 Tensor
            image_tensor = image_tensor.unsqueeze(0).unsqueeze(0)  # 增加 batch 和 channel 維度
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            continue

        with torch.no_grad():
            outputs = model(image_tensor)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            predicted_class = torch.argmax(probs, dim=1).item()
            predictions.append(predicted_class)
            probabilities.append(probs.numpy().flatten())

    return predictions, probabilities

In [ ]:
# 主程式示例
if __name__ == "__main__":
    # 模型路徑
    model_path = './best_model_hung.pth'
    model = load_model_for_prediction(model_path)

    # 測試圖片路徑
    image_paths = [
        'C:\\Users\\user\\Desktop\\Leet\\finance\\data\\pred_5_to_5\\5_5\\1101_20040105.png',  
        'C:\\Users\\user\\Desktop\\Leet\\finance\\data\\pred_5_to_5\\5_5\\5381_20070823.png',
    ]

    predictions, probabilities = predict_new_data(model, image_paths)

    for img, pred, prob in zip(image_paths, predictions, probabilities):
        print(f"Image: {img}")
        print(f"Predicted Class: {pred}")
        print(f"Probabilities: {prob}")

Image: C:\Users\user\Desktop\Leet\finance\data\pred_5_to_5\5_5\5381_20070712.png
Predicted Class: 0
Probabilities: [0.5258791  0.47412086]
Image: C:\Users\user\Desktop\Leet\finance\data\pred_5_to_5\5_5\5381_20070823.png
Predicted Class: 1
Probabilities: [0.4781194 0.5218806]
